In [1]:
# coding=utf-8
import nltk
from nltk.corpus import stopwords
import nltk.data
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import zeros
import os
import re
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
import itertools
# import sys 
# reload(sys)
# sys.setdefaultencoding("utf-8")

### Define some useful function for the next steps

In [2]:
def keyFunc(afilename):
    #A key function used to sort the file list
    m = re.search('(?<=Episode)\d{2}', afilename)
    return m.group(0)

def untokenize(words):
    """
    Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    #ref : https://github.com/commonsense/metanl/blob/master/metanl/token_utils.py
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .', '...')
    step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
        "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    return step6.strip()

def get_frequent_word(tfidf, episod):
    X = tfidf.transform([episod])
    tfidf_table = X.todense().tolist()[0]
    frequent_word = []
    candidate_number = 20
    for word in sorted(zip(tfidf_table,tfidf.get_feature_names()), reverse=True)[:candidate_number]:
        frequent_word.append(word[1])
    return frequent_word

def make_sentence (untokenized):
    #ref :https://github.com/thavelick/summarize/blob/master/summarize.py
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    normal_sentences = sent_detector.tokenize(untokenized) 
    working_sentences = [sentence.lower() for sentence in normal_sentences]
    return working_sentences, normal_sentences


def get_output_sentence(most_frequent_words, working_sentences, normal_sentences, num_phrase):
    temp_sentences = []
    output_sentences = []
    for word in most_frequent_words:
        for i in range(len(working_sentences)):
            if (word in working_sentences[i] and normal_sentences[i] not in temp_sentences):
                temp_sentences.append(normal_sentences[i]) 
    counter = zeros(len(temp_sentences))
    for i in range(len(temp_sentences)):
        for word in most_frequent_words:
            if word in temp_sentences[i]:
                counter[i]+=1        
    for sentence in sorted(zip(counter,temp_sentences), reverse=True)[:num_phrase]:
        output_sentences.append(sentence[1])
    return output_sentences

def reorder_sentences(output_sentences, original):
    ordered_output=[]
    for sentence in original:
        if sentence in output_sentences:
            ordered_output.append(sentence)
    return ordered_output

### Read the files and prepare the corpus for the next step

In [3]:
dir_path = './sous_titre/'
file_list = sorted(os.listdir(dir_path), key=keyFunc)
collection=[] #document collection
for file in file_list:
    file_path = os.path.join(dir_path, file)
    if os.path.isfile(file_path):
        with open(file_path,"rt") as f:
            lines=f.readlines()
            episod=[]
            for x in lines:
                episod.append(x.split(' ')[4])
        collection.append(episod)
        
untokenized_collection=[]
        
for episod in range(len(collection)):
    untokenized_collection.append(untokenize(collection[episod]))


print (untokenized_collection[0])


UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 56: ordinal not in range(128)

### Calculer le tf-idf and prepare the model

In [ ]:
dir_path = './summary/'
f_list = sorted(os.listdir(dir_path))
file_list=[]
file_list.append(f_list[0])
file_list.extend(f_list[9:])
file_list.extend(f_list[1:9])
r_collection=[] #document collection
for file in file_list:
    file_path = os.path.join(dir_path, file)
    if os.path.isfile(file_path):
        with open(file_path,"rt") as f:
            lines=f.readlines()
            episod=[]
            for x in lines:
                episod.append(x)
        r_collection.append(episod)
        
r_untokenized_collection=[]
        
for episod in range(len(r_collection)):
    r_untokenized_collection.append(untokenize(r_collection[episod]))


print (r_untokenized_collection[0])

In [ ]:
oral_words = ['oh', 'yes', 'no', 'hi', 'okay', 'uh', 'okay', 'bye', 'sorry', 'well', 'think','know','going','yeah'] #A compléter dans les tests
stop_words = stopwords.words('english') + list(punctuation) + oral_words
tfidf = TfidfVectorizer(stop_words=stop_words)
tfidf.fit(untokenized_collection[episod] for episod in range(len(untokenized_collection)))
print ('Model prepared! You can make your summarize now!')


### for changing the episod, just modify the index of untokenized_collection

In [ ]:
most_frequent_words = get_frequent_word(tfidf, untokenized_collection[0])
working_sentences, normal_sentences = make_sentence (untokenized_collection[0])
num_sentences = 15
out=get_output_sentence(most_frequent_words, working_sentences, normal_sentences,num_sentences)
print (reorder_sentences(out, normal_sentences))

In [ ]:
def _get_ngrams(n, text):
    """Calcualtes n-grams.
    Args:
      n: which n-grams to calculate
      text: An array of tokens
    Returns:
      A set of n-grams
    """
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set


def _split_into_words(sentences):
    """Splits multiple sentences into words and flattens the result"""
    return list(itertools.chain(*[_.split(" ") for _ in sentences]))
#     return sentences.split(" ")

def _get_word_ngrams(n, sentences):
    """Calculates word n-grams for multiple sentences.
    """
    assert len(sentences) > 0
    assert n > 0

    words = _split_into_words(sentences)
    return _get_ngrams(n, words)

def rouge_n(evaluated_sentences, reference_sentences, n=2):
    """
    Computes ROUGE-N of two text collections of sentences.
    Sourece: http://research.microsoft.com/en-us/um/people/cyl/download/
    papers/rouge-working-note-v1.3.1.pdf
    Args:
      evaluated_sentences: The sentences that have been picked by the
                           summarizer
      reference_sentences: The sentences from the referene set
      n: Size of ngram.  Defaults to 2.
    Returns:
      A tuple (f1, precision, recall) for ROUGE-N
    Raises:
      ValueError: raises exception if a param has len <= 0
    """
    if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
        raise ValueError("Collections must contain at least 1 sentence.")

    evaluated_ngrams = _get_word_ngrams(n, evaluated_sentences)
    reference_ngrams = _get_word_ngrams(n, reference_sentences)
#     print (evaluated_ngrams)
    reference_count = len(reference_ngrams)
    evaluated_count = len(evaluated_ngrams)

    # Gets the overlapping ngrams between evaluated and reference
    overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
    overlapping_count = len(overlapping_ngrams)

    # Handle edge case. This isn't mathematically correct, but it's good enough
    if evaluated_count == 0:
        precision = 0.0
    else:
        precision = overlapping_count / evaluated_count

    if reference_count == 0:
        recall = 0.0
    else:
        recall = overlapping_count / reference_count

    f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))

    return {"Precision": precision, "Recall": recall, "F1-score": f1_score,}

In [ ]:
h1="he is a good man."
r1="she is a good woman."
h1=[h1]
r1=[r1]
r_1=rouge_n(h1,r1,1)
r_2=rouge_n(h1,r1,2)
print(r_1)
print(r_2)

In [ ]:
def _len_lcs(x, y):
    """
    Returns the length of the Longest Common Subsequence between sequences x
    and y.
    Source: http://www.algorithmist.com/index.php/Longest_Common_Subsequence
    Args:
      x: sequence of words
      y: sequence of words
    Returns
      integer: Length of LCS between x and y
    """
    table = _lcs(x, y)
    n, m = len(x), len(y)
    return table[n, m]


def _lcs(x, y):
    """
    Computes the length of the longest common subsequence (lcs) between two
    strings. The implementation below uses a DP programming algorithm and runs
    in O(nm) time where n = len(x) and m = len(y).
    Source: http://www.algorithmist.com/index.php/Longest_Common_Subsequence
    Args:
      x: collection of words
      y: collection of words
    Returns:
      Table of dictionary of coord and len lcs
    """
    n, m = len(x), len(y)
    table = dict()
    for i in range(n + 1):
        for j in range(m + 1):
            if i == 0 or j == 0:
                table[i, j] = 0
            elif x[i - 1] == y[j - 1]:
                table[i, j] = table[i - 1, j - 1] + 1
            else:
                table[i, j] = max(table[i - 1, j], table[i, j - 1])
    return table


def _recon_lcs(x, y):
    """
    Returns the Longest Subsequence between x and y.
    Source: http://www.algorithmist.com/index.php/Longest_Common_Subsequence
    Args:
      x: sequence of words
      y: sequence of words
    Returns:
      sequence: LCS of x and y
    """
    i, j = len(x), len(y)
    table = _lcs(x, y)

    def _recon(i, j):
        """private recon calculation"""
        if i == 0 or j == 0:
            return []
        elif x[i - 1] == y[j - 1]:
            return _recon(i - 1, j - 1) + [(x[i - 1], i)]
        elif table[i - 1, j] > table[i, j - 1]:
            return _recon(i - 1, j)
        else:
            return _recon(i, j - 1)

    recon_tuple = tuple(map(lambda x: x[0], _recon(i, j)))
    return recon_tuple




def _union_lcs(evaluated_sentences, reference_sentence, prev_union=None):
    """
    Returns LCS_u(r_i, C) which is the LCS score of the union longest common
    subsequence between reference sentence ri and candidate summary C.
    For example:
    if r_i= w1 w2 w3 w4 w5, and C contains two sentences: c1 = w1 w2 w6 w7 w8
    and c2 = w1 w3 w8 w9 w5, then the longest common subsequence of r_i and c1
    is "w1 w2" and the longest common subsequence of r_i and c2 is "w1 w3 w5".
    The union longest common subsequence of r_i, c1, and c2 is "w1 w2 w3 w5"
    and LCS_u(r_i, C) = 4/5.
    Args:
      evaluated_sentences: The sentences that have been picked by the
                           summarizer
      reference_sentence: One of the sentences in the reference summaries
    Returns:
      float: LCS_u(r_i, C)
    ValueError:
      Raises exception if a param has len <= 0
    """
    if prev_union is None:
        prev_union = set()

    if len(evaluated_sentences) <= 0:
        raise ValueError("Collections must contain at least 1 sentence.")

    lcs_union = prev_union
    prev_count = len(prev_union)
    reference_words = _split_into_words([reference_sentence])

    combined_lcs_length = 0
    for eval_s in evaluated_sentences:
        evaluated_words = _split_into_words([eval_s])
        lcs = set(_recon_lcs(reference_words, evaluated_words))
        combined_lcs_length += len(lcs)
        lcs_union = lcs_union.union(lcs)

    new_lcs_count = len(lcs_union) - prev_count
    return new_lcs_count, lcs_union


def rouge_l_summary_level(evaluated_sentences, reference_sentences):
    """
    Computes ROUGE-L (summary level) of two text collections of sentences.
    http://research.microsoft.com/en-us/um/people/cyl/download/papers/
    rouge-working-note-v1.3.1.pdf
    Calculated according to:
    R_lcs = SUM(1, u)[LCS<union>(r_i,C)]/m
    P_lcs = SUM(1, u)[LCS<union>(r_i,C)]/n
    F_lcs = ((1 + beta^2)*R_lcs*P_lcs) / (R_lcs + (beta^2) * P_lcs)
    where:
    SUM(i,u) = SUM from i through u
    u = number of sentences in reference summary
    C = Candidate summary made up of v sentences
    m = number of words in reference summary
    n = number of words in candidate summary
    Args:
      evaluated_sentences: The sentences that have been picked by the
                           summarizer
      reference_sentence: One of the sentences in the reference summaries
    Returns:
      A float: F_lcs
    Raises:
      ValueError: raises exception if a param has len <= 0
    """
    if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
        raise ValueError("Collections must contain at least 1 sentence.")

    # total number of words in reference sentences
    m = len(set(_split_into_words(reference_sentences)))

    # total number of words in evaluated sentences
    n = len(set(_split_into_words(evaluated_sentences)))

    # print("m,n %d %d" % (m, n))
    union_lcs_sum_across_all_references = 0
    union = set()
    for ref_s in reference_sentences:
        lcs_count, union = _union_lcs(evaluated_sentences,
                                      ref_s,
                                      prev_union=union)
        union_lcs_sum_across_all_references += lcs_count

    llcs = union_lcs_sum_across_all_references
    r_lcs = llcs / m
    p_lcs = llcs / n
    beta = p_lcs / (r_lcs + 1e-12)
    num = (1 + (beta**2)) * r_lcs * p_lcs
    denom = r_lcs + ((beta**2) * p_lcs)
    f_lcs = num / (denom + 1e-12)
    return {"Precision": p_lcs, "Recall": r_lcs, "F1-score": f_lcs}

In [ ]:
r_l=rouge_l_summary_level(h1,r1)
print(r_l)

In [ ]:
test=reorder_sentences(out,normal_sentences)
print(test)

reference_text=r_untokenized_collection[0]
t1,t2=make_sentence(reference_text)

print(t1)

In [ ]:
s=""
for i in t1:
    s+=i
s=[s]

s1=""
for i in test:
    s1+=i

s1=[s1]

In [ ]:
for hyp, ref in zip(s, s1):
        hyp = [" ".join(_.split()) for _ in hyp.split(".") if len(_) > 0]
        ref = [" ".join(_.split()) for _ in ref.split(".") if len(_) > 0]

In [ ]:
print(rouge_n(hyp,ref,1))
print(rouge_n(hyp,ref,2))
print(rouge_l_summary_level(hyp,ref))

呜啦啦啦火车笛，做projet很忙滴。。大家唱起来！俗话说的好，彩蛋最后找，要是没找到。。。。。。。。。。。。。
















说明你小子根本没仔细看劳资辛辛苦苦写的代码！